<a href="https://colab.research.google.com/github/manisshapande/llm_engineering/blob/main/ChatGPT_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ChatGPT Interface ##

In [ ]:

#Imports

import os
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import Markdown, display, update_display, clear_output
from openai import OpenAI

In [ ]:
# Loading, Setting model and open api key
load_dotenv()
api_key = os.getenv('OPEN_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
  print("We are goos with API Key for further proceeding")
else:
  print("There is a problem with your api key. Please check or contact your admin")

MODEL = "gpt-4o-mini"

In [ ]:
#Create chatgpt interface

class ChatInterface:
    def __init__(self, api_key, model, system_message="You are a helpful assistant. You can format your responses using Markdown."):
        self.openai = OpenAI(api_key=api_key)
        self.api_key = api_key
        self.model = model
        #self.system_message = system_message
        self.conversation_history = [{"role": "system", "content": system_message}]

        # Initialize widgets
        self.chat_area = widgets.Output()
        self.input_box = widgets.Text(placeholder="Enter your message here...", layout=widgets.Layout(flex="1"))
        self.send_button = widgets.Button(description="Send", button_style="primary")

        # Attach event handler
        self.send_button.on_click(self._handle_send_click)

        # Layout widgets
        self.layout = self._build_layout()

    def display(self):
        """Display the entire interface."""
        display(self.layout)

    def _build_layout(self):
        """Construct and return the layout of the interface."""
        input_area = widgets.HBox([self.input_box, self.send_button])
        input_area.layout.align_items = "center"
        return widgets.VBox([self.chat_area, input_area])

    def _handle_send_click(self, _):
        """Handle the click event for the send button."""
        user_message = self.input_box.value.strip()
        if user_message:
            self.display_message("You", user_message)
            self.conversation_history.append({"role": "user", "content": user_message})
            self.input_box.value = ""

            try:
                response = self.openai.chat.completions.create(
                    model=self.model,
                    messages=self.conversation_history
                )
                assistant_message = response.choices[0].message.content.strip()
                self.conversation_history.append({"role": "assistant", "content": assistant_message})
                self.display_message("ChatGPT", assistant_message)
            except Exception as e:
                self.display_message("Error", f"An error occurred: {str(e)}")

    def display_message(self, sender, message):
        """Add a message to the chat area."""
        self.chat_area.append_display_data(Markdown(f"**{sender}:**\n{message}"))


In [ ]:
chat_interface = ChatInterface(api_key,MODEL)
chat_interface.display()